## Access the database

In [25]:
# define parameters
db_name = 'dbsprojekt' # db should already be created locally
user = 'postgres'
password = 'password5432'

import_done = True # nach dem ersten Ausführen auf True setzen

In [1]:
import pandas as pd
# import numpy as np
import psycopg2

In [3]:
# connect to database
try:
    print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(host = 'localhost', database = db_name, user = user, password = password)
    
    cur = conn.cursor()
    
    print('connected')
    
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connecting to the PostgreSQL database...
connected


## Read in data files and create tables

In [26]:
# read files
f1 = open('literature_dataset.csv')
f2 = open('journal_dataset.csv')

# list of all attributes in each table
attr_list_lit = f1.readline().replace('\n', '').split(sep = ',')
attr_list_jif = f2.readline().replace('\n', '').split(sep = ',')

if not import_done:
    try:
        # create table for literature dataset
        # get list of attributes
        attr_list = ''
        for i in attr_list_lit:
            attr_list += '{} varchar, '.format(i)
        attr_list = attr_list[:-2] # entfernt ', ' am Ende der Liste

        # create literature table & import data
        sql_create = 'CREATE TABLE literature ( {} );'.format(attr_list)
        print(sql_create)
        cur.execute(sql_create)
        cur.copy_expert("""COPY literature FROM STDIN WITH CSV HEADER DELIMITER AS ','""", f1)
        
        # create table for journal dataset
        # get list of attributes        
        attr_list = ''
        for i in attr_list_jif:
            attr_list += '{} varchar, '.format(i)
        attr_list = attr_list[:-2]
        
        # create impact factors table & import data
        sql_create = 'CREATE TABLE journals ( {} );'.format(attr_list)
        print(sql_create)
        cur.execute(sql_create)
        cur.copy_expert("""COPY journals FROM STDIN WITH CSV HEADER DELIMITER AS ','""", f2)
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

## Execute queries

In [27]:
sql_showtable = """
    SELECT *
    FROM literature;
    """

# query to execute
sql_query = sql_showtable

In [28]:
try:
    cur.execute(sql_query)
    some_results = cur.fetchmany(10)
    df = pd.DataFrame(data = some_results, columns = attr_list_lit)

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

## Display results

In [29]:
df

,Titel,Autor,Jahr,Journal,Typ,DOI
0,implementation of local area vr environment us...,"kim, soo-kyun; lee, chang-hee; kim, sun-jeong;...",2020,intelligent automation and soft computing,None,10.31209/2019.100000131
1,distinction between real faces and photos by a...,"lee, byong kwon; lee, yang sun",2020,intelligent automation and soft computing,None,10.31209/2019.100000134
2,cognitive load and performance in immersive vi...,"frederiksen, joakim grant; sorensen, stine may...",2020,surgical endoscopy and other interventional te...,None,10.1007/s00464-019-06887-8
3,tracking attacks on virtual reality systems,"rafique, muhammad usman; cheung, sen-ching s.",2020,ieee consumer electronics magazine,None,10.1109/MCE.2019.2953741
4,advantages and limits of virtual reality in le...,"checa, david; bustillo, andres",2020,virtual reality,None,10.1007/s10055-019-00389-7
5,the relationship between disgust sensitivity a...,"ammann, jeanine; hartmann, christina; peterhan...",2020,food quality and preference,None,10.1016/j.foodqual.2019.103833
6,cybersickness in virtual reality head-mounted ...,"curry, christopher; li, ruixuan; peterson, nic...",2020,international journal of human-computer intera...,None,10.1080/10447318.2020.1726108
7,relationship between ocular fatigue and use of...,"lee, sang hyeok; kim, martha; kim, hyosun; par...",2020,journal of the korean ophthalmological society,None,10.3341/jkos.2020.61.2.125
8,depth perception in shuffleboard: depth cues e...,"ping, jiamin; weng, dongdong; liu, yue; wang, ...",2020,journal of the society for information display,None,10.1002/jsid.840
9,the challenges of using head mounted virtual r...,"fransson, goran; holmberg, jorgen; westelius, ...",2020,education and information technologies,None,10.1007/s10639-020-10119-1


In [23]:
# can use after getting an error
# conn.rollback()

## Close the connection

In [30]:
try:
    conn.commit()

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

In [31]:
cur.close()
print('connection closed')

connection closed
